In [5]:
import os
import sys
import subprocess

# 현재 설치된 패키지 목록을 가져옴
installed_packages = subprocess.check_output([sys.executable, '-m', 'pip', 'list']).decode('utf-8')

# 'llamaindex'를 포함하는 패키지를 식별함
for package in installed_packages.split('\n'):
    if 'llama' in package:
        package_name = package.split()[0]  # 패키지 이름을 추출함
        # 해당 패키지를 삭제함
        print(f"Uninstalling {package_name}...")
        os.system(f"pip uninstall -y {package_name}")

print("Done.")

Done.


In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import torch
from transformers import AutoTokenizer, TextStreamer, GenerationConfig
from attention_sinks import AutoModelForCausalLM

device = "cuda"
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # for efficiency:
    device_map=device,
    # load_in_4bit=True
    torch_dtype=torch.float16,
    # `attention_sinks`-specific arguments:
    attention_sink_size=4,
    attention_sink_window_size=252, # <- Low for the sak of faster generation
)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

/home/tbelldev/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.35s/it]
[Attention Sinks] Injected Position Shifting into 32 attention classes.
[Attention Sinks] Injected Attention Sink KV Cache into 1 model class.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [54]:
# Setup a prompt 
prompt = "### User: Generate 100 fake spam SMS text message for text data augmentation. for EXAMPLE : \
[Prize Notification Spam]\
Congratulations! You've won a $1,000 prize in our special event. To claim your prize, click this link now: www.sampleprizelink.com/claim or call +13526450.\
You're our lucky $500 gift card winner! To get your gift, visit www.winnernotice.net immediately. Promo ends today!\
[Parcel/Delivery Spam]\
Your package is pending delivery. To complete delivery, pay the shipping fee via this link: www.fakeparceltrack.com/pay. For queries, contact +12345316378.\
We couldn't deliver your parcel today. Reschedule now at www.misseddelivery.com or it will be returned to sender.\
[Banking/Financial Fraud Spam]\
Alert: Your Bank XYZ account has a security issue. Secure your account by visiting www.bankxyz-security.com. Call us at +113267892 for support.\
Important: We've noticed unusual activity in your account. Confirm your identity at www.fakebankalert.com to avoid suspension.\
[Fake Alert Messages]\
Critical Warning: Your software license has expired. Renew immediately by clicking here: www.softwarelicenseupdate.com.\
Security Alert: Your computer has been infected with a virus. Click here for a free cleanup: www.computercleanup.com.\
[Fake Employment/Job Offer Spam]\
You are the perfect candidate for our new position! Send your CV now and book your interview slot at www.joboffer.com.\
Immediate hiring! Apply now for an exclusive job opportunity with us. Visit www.careeropportunity.net for more information.\
[Insurance Advertisement Spam]\
Protect your family with affordable insurance rates! Check out the best offer now by clicking www.insurecheap.info.\
Life insurance plans at a low cost just for you! Explore more at www.lifeinsurepromo.com.\
[Fake Surveys/Polls]\
Invest just a few minutes to complete this survey and win a $50 voucher! Participate now at www.surveyreward.com.\
Your opinion matters! Fill out our quick survey and you could win $100 in cash! Click here: www.surveyprize.net.\
[Romance/Dating Scams]\
I want to meet you! Check out my profile here and let's chat: www.fakeromance.com.\
Find your soulmate today! Sign up at www.loveconnection.net to start your love story.\
[Healthcare/Product Sales Spam]\
Rediscover your health with our innovative wellness supplement, now at a discount! Visit www.healthboostnow.com.\
Exclusive offer on our new health product! Improve your life quality today: www.newhealthproduct.com.\
[Subscription/Service Cancellation Spam]\
Your subscription is about to expire. Continue enjoying our service by renewing here: www.subscribenow.com.\
Don’t miss out on our exclusive content! Your subscription ends soon. Renew now at www.renewsubscription.com.\
RULES : 1. don't add hashtag 2.only generate spam SMS text message 3.message must include phone number, addr or price or links.\
          ### Assistant:"
# Pass the prompt to the tokenizer
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
# Setup the text streamer 
streamer = TextStreamer(tokenizer, skip_prompt=False, skip_special_tokens=True)

In [55]:
# Actually run the thing
output = model.generate(**inputs, streamer=streamer, 
                        use_cache=True, max_new_tokens=float('inf'))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### User: Generate 100 fake spam SMS text message for text data augmentation. for EXAMPLE : [Prize Notification Spam]Congratulations! You've won a $1,000 prize in our special event. To claim your prize, click this link now: www.sampleprizelink.com/claim or call +13526450.You're our lucky $500 gift card winner! To get your gift, visit www.winnernotice.net immediately. Promo ends today![Parcel/Delivery Spam]Your package is pending delivery. To complete delivery, pay the shipping fee via this link: www.fakeparceltrack.com/pay. For queries, contact +12345316378.We couldn't deliver your parcel today. Reschedule now at www.misseddelivery.com or it will be returned to sender.[Banking/Financial Fraud Spam]Alert: Your Bank XYZ account has a security issue. Secure your account by visiting www.bankxyz-security.com. Call us at +113267892 for support.Important: We've noticed unusual activity in your account. Confirm your identity at www.fakebankalert.com to avoid suspension.[Fake Alert Messages]Cri

In [35]:
# Covert the output tokens back to text 
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
output_text

'### User: Generate 10 fake spam SMS text message for text data augmentation.          ### Assistant: Sure, here are 10 fake spam SMS text messages for text data augmentation:\n\n1. "Win 1000$ by clicking this link now! 💸📲"\n2. "Get rich quick with our investment scheme! 💰📲"\n3. "Free pizza tonight! 🍕📲"\n4. "Meet your soulmate on our dating app! 💘📲"\n5. "Get exclusive access to our VIP club! 💼📲"\n6. "Win a trip to Hawaii! 🌴📲"\n7. "Get a free tattoo! 💅📲"\n8. "Get a discount on your next purchase! 🛍️📲"\n9. "Win a brand new car! 🚗📲"\n10. "Get a free gift card! 🎁📲"'

In [47]:
from llama_index.core.prompts.prompts import PromptTemplate
# Create a system prompt 
system_prompt = """
You are a helpful text data generator. data that you generate is realistic and plausible short SMS spam TEXT messages including fake phone number, addr or price. 
for EXAMPLE : 
[Prize Notification Spam]
"Congratulations! You've won a $1,000 prize in our special event. To claim your prize, click this link now: www.sampleprizelink.com/claim or call +13526450."
"You're our lucky $500 gift card winner! To get your gift, visit www.winnernotice.net immediately. Promo ends today!"
[Parcel/Delivery Spam]
"Your package is pending delivery. To complete delivery, pay the shipping fee via this link: www.fakeparceltrack.com/pay. For queries, contact +12345316378."
"We couldn't deliver your parcel today. Reschedule now at www.misseddelivery.com or it will be returned to sender."
[Banking/Financial Fraud Spam]
"Alert: Your Bank XYZ account has a security issue. Secure your account by visiting www.bankxyz-security.com. Call us at +113267892 for support."
"Important: We've noticed unusual activity in your account. Confirm your identity at www.fakebankalert.com to avoid suspension."
[Fake Alert Messages]
"Critical Warning: Your software license has expired. Renew immediately by clicking here: www.softwarelicenseupdate.com."
"Security Alert: Your computer has been infected with a virus. Click here for a free cleanup: www.computercleanup.com."
[Fake Employment/Job Offer Spam]
"You are the perfect candidate for our new position! Send your CV now and book your interview slot at www.joboffer.com."
"Immediate hiring! Apply now for an exclusive job opportunity with us. Visit www.careeropportunity.net for more information."
[Insurance Advertisement Spam]
"Protect your family with affordable insurance rates! Check out the best offer now by clicking www.insurecheap.info."
"Life insurance plans at a low cost just for you! Explore more at www.lifeinsurepromo.com."
[Fake Surveys/Polls]
"Invest just a few minutes to complete this survey and win a $50 voucher! Participate now at www.surveyreward.com."
"Your opinion matters! Fill out our quick survey and you could win $100 in cash! Click here: www.surveyprize.net."
[Romance/Dating Scams]
"I want to meet you! Check out my profile here and let's chat: www.fakeromance.com."
"Find your soulmate today! Sign up at www.loveconnection.net to start your love story."
[Healthcare/Product Sales Spam]
"Rediscover your health with our innovative wellness supplement, now at a discount! Visit www.healthboostnow.com."
"Exclusive offer on our new health product! Improve your life quality today: www.newhealthproduct.com."
[Subscription/Service Cancellation Spam]
"Your subscription is about to expire. Continue enjoying our service by renewing here: www.subscribenow.com."
"Don’t miss out on our exclusive content! Your subscription ends soon. Renew now at www.renewsubscription.com.

RULES : 1. don't add hashtag 2.only generate spam SMS text message 3.message must include phone number, addr or price or links.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + system_prompt + "<</SYS>>\n\n{query_str}[/INST] "
)

In [49]:
type(query_wrapper_prompt)

llama_index.core.prompts.base.PromptTemplate

In [57]:
from llama_index.legacy.llms import HuggingFaceLLM
llm = HuggingFaceLLM(context_window=4096,
                    max_new_tokens=256,
                    system_prompt=system_prompt,
                    # query_wrapper_prompt=query_wrapper_prompt,
                    generate_kwargs={},
                    model_kwargs={"n_gpu_layers": 1},
                    model=model,
                    callback_manager=None,
                    tokenizer=tokenizer)

In [42]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.core.readers import Document
documents = SimpleDirectoryReader(
    # input_files = ["documents/survey_on_llms.pdf"]
    input_files = ["documents/spam_messages.pdf"]
).load_data()


documents = Document(text = "\n\n".join([doc.text for doc in documents]))
type(documents)

llama_index.core.schema.Document

In [43]:
# Putting all it together
import os
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import VectorStoreIndex, ServiceContext, StorageContext, load_index_from_storage
def get_build_index(documents,llm,embed_model="local:BAAI/bge-small-en-v1.5",sentence_window_size=3,save_dir="./vector_index/index"):

  node_parser = SentenceWindowNodeParser(
      window_size = sentence_window_size,
      window_metadata_key = "window",
      original_text_metadata_key = "original_text"
  )

  sentence_context = ServiceContext.from_defaults(
      llm = llm,
      embed_model= embed_model,
      node_parser = node_parser,
  )

  if not os.path.exists(save_dir):
        # create and load the index
        index = VectorStoreIndex.from_documents(
            [documents], service_context=sentence_context
        )
        index.storage_context.persist(persist_dir=save_dir)
  else:
      # load the existing index
      index = load_index_from_storage(
          StorageContext.from_defaults(persist_dir=save_dir),
          service_context=sentence_context,
      )

  return index

In [44]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor, SentenceTransformerRerank
def get_query_engine(sentence_index, similarity_top_k=6, rerank_top_n=2):
  postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
  rerank = SentenceTransformerRerank(
      top_n=rerank_top_n, model="BAAI/bge-reranker-base"
  )
  engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
  )

  return engine

In [58]:
vector_index = get_build_index(documents=documents, llm=llm, embed_model="local:BAAI/bge-small-en-v1.5", sentence_window_size=3, save_dir="./vector_store/index")
query_engine = get_query_engine(sentence_index=vector_index, similarity_top_k=6, rerank_top_n=2)

/tmp/ipykernel_1180583/3545213728.py:13: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(


In [59]:
query = "Generate 10 realistic and plausible short spam messages"
response = query_engine.query(query)
print(response)

ValueError: Cannot use llm_completion_callback on an instance without a callback_manager attribute.